In [1]:
%%pyspark
df = spark.read.load('abfss://wwi-02@asadatalake1xtr2e3.dfs.core.windows.net/sale-small/Year=2019/Quarter=Q4/Month=12/Day=20191231/sale-small-20191231-snappy.parquet', format='parquet')
display(df.limit(10))

StatementMeta(sparkpool2, 4, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 50442521-ed86-4f4f-a286-ab964da0d00c)

In [2]:
datalake = 'asadatalake1xtr2e3'

StatementMeta(sparkpool2, 4, 3, Finished, Available)

In [3]:
df.printSchema()

StatementMeta(sparkpool2, 4, 4, Finished, Available)

root
 |-- TransactionId: string (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- ProductId: short (nullable = true)
 |-- Quantity: byte (nullable = true)
 |-- Price: decimal(38,18) (nullable = true)
 |-- TotalAmount: decimal(38,18) (nullable = true)
 |-- TransactionDate: integer (nullable = true)
 |-- ProfitAmount: decimal(38,18) (nullable = true)
 |-- Hour: byte (nullable = true)
 |-- Minute: byte (nullable = true)
 |-- StoreId: short (nullable = true)



In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

profitByDateProduct = (df.groupBy("TransactionDate","ProductId")
    .agg(
        sum("ProfitAmount").alias("(sum)ProfitAmount"),
        round(avg("Quantity"), 4).alias("(avg)Quantity"),
        sum("Quantity").alias("(sum)Quantity"))
    .orderBy("TransactionDate"))
display(profitByDateProduct.limit(100))

StatementMeta(sparkpool2, 4, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2a4a5d4d-ff9c-4649-a9be-468d08045c72)

In [5]:
df = (spark.read \
        .option('inferSchema', 'true') \
        .json('abfss://wwi-02@' + datalake + '.dfs.core.windows.net/online-user-profiles-02/*.json', multiLine=True)
    )

df.printSchema()

StatementMeta(sparkpool2, 4, 6, Finished, Available)

root
 |-- topProductPurchases: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- itemsPurchasedLast12Months: long (nullable = true)
 |    |    |-- productId: long (nullable = true)
 |-- visitorId: long (nullable = true)



In [6]:
# create a view called user_profiles
df.createOrReplaceTempView("user_profiles")

StatementMeta(sparkpool2, 4, 7, Finished, Available)

In [7]:
%%sql

SELECT * FROM user_profiles LIMIT 10

StatementMeta(sparkpool2, 4, 8, Finished, Available)

<Spark SQL result set with 10 rows and 2 fields>

In [8]:
from pyspark.sql.functions import udf, explode

flat=df.select('visitorId',explode('topProductPurchases').alias('topProductPurchases_flat'))
flat.show(100)

StatementMeta(sparkpool2, 4, 9, Finished, Available)

+---------+------------------------+
|visitorId|topProductPurchases_flat|
+---------+------------------------+
|   117000|              {13, 3623}|
|   117000|               {5, 2321}|
|   117001|               {93, 713}|
|   117001|              {19, 2144}|
|   117001|              {30, 1094}|
|   117001|              {82, 3223}|
|   117001|              {42, 3328}|
|   117001|              {62, 2926}|
|   117001|              {63, 2651}|
|   117001|               {39, 341}|
|   117001|              {85, 4841}|
|   117001|              {67, 4289}|
|   117001|              {42, 1264}|
|   117001|              {43, 3608}|
|   117001|               {14, 504}|
|   117001|              {97, 2649}|
|   117001|              {44, 2873}|
|   117001|               {7, 4491}|
|   117001|              {70, 3623}|
|   117001|               {4, 4035}|
|   117001|              {45, 3576}|
|   117002|               {25, 478}|
|   117002|              {15, 2515}|
|   117002|               {34, 209}|
|

In [9]:
topPurchases = (flat.select('visitorId','topProductPurchases_flat.productId','topProductPurchases_flat.itemsPurchasedLast12Months')
    .orderBy('visitorId'))

topPurchases.show(100)

StatementMeta(sparkpool2, 4, 10, Finished, Available)

+---------+---------+--------------------------+
|visitorId|productId|itemsPurchasedLast12Months|
+---------+---------+--------------------------+
|    80000|     4024|                        35|
|    80000|      290|                        83|
|    80000|     3122|                        33|
|    80000|     4206|                        21|
|    80000|     2481|                        63|
|    80000|       93|                        86|
|    80000|      102|                        42|
|    80000|     1330|                        78|
|    80000|     2069|                        93|
|    80000|     2859|                        54|
|    80000|     3794|                        90|
|    80000|     4136|                        30|
|    80000|     4074|                        48|
|    80000|     4198|                        92|
|    80000|     3538|                        65|
|    80000|     2488|                        31|
|    80000|      291|                        49|
|    80000|     3270

In [10]:
# Let's order by the number of items purchased in the last 12 months
sortedTopPurchases = topPurchases.orderBy("itemsPurchasedLast12Months")

display(sortedTopPurchases.limit(100))

StatementMeta(sparkpool2, 4, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7512d721-2fdb-4521-8427-de83290cb27b)

In [12]:
sortedTopPurchases = (topPurchases
    .orderBy( col("itemsPurchasedLast12Months").desc() ))

display(sortedTopPurchases.limit(100))

StatementMeta(sparkpool2, 4, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 90e4543b-ce9e-4999-b5f4-b0bc4561d4f7)

In [13]:
groupedTopPurchases = (sortedTopPurchases.select("visitorId")
    .groupBy("visitorId")
    .agg(count("*").alias("total"))
    .orderBy("visitorId") )

display(groupedTopPurchases.limit(100))

StatementMeta(sparkpool2, 4, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, d65c99d7-dc47-4176-92bc-dc8d9eecb927)

In [14]:
groupedTopPurchases = (sortedTopPurchases.select("visitorId","itemsPurchasedLast12Months")
    .groupBy("visitorId")
    .agg(sum("itemsPurchasedLast12Months").alias("totalItemsPurchased"))
    .orderBy("visitorId") )

display(groupedTopPurchases.limit(100))

StatementMeta(sparkpool2, 4, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 345ca825-9952-49a8-b50e-0c570787698f)

In [15]:
# Create a temporary view for top purchases so we can load from Scala
topPurchases.createOrReplaceTempView("top_purchases")

StatementMeta(sparkpool2, 4, 16, Finished, Available)

In [16]:
%%spark
// Make sure the name of the dedcated SQL pool (SQLPool01 below) matches the name of your SQL pool.
val df = spark.sqlContext.sql("select * from top_purchases")
df.write.synapsesql("SQLPool01.wwi.TopPurchases", Constants.INTERNAL)

StatementMeta(sparkpool2, 4, 18, Finished, Available)

df: org.apache.spark.sql.DataFrame = [visitorId: bigint, productId: bigint ... 1 more field]


In [17]:
dfsales = spark.read.load('abfss://wwi-02@' + datalake + '.dfs.core.windows.net/sale-small/Year=2019/Quarter=Q4/Month=12/*/*.parquet', format='parquet')
display(dfsales.limit(10))

StatementMeta(sparkpool2, 4, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, ed5ef3be-8e7c-46f5-9b2d-7e23ecd78a60)

In [18]:
%%spark
// Make sure the name of the SQL pool (SQLPool01 below) matches the name of your SQL pool.
val df2 = spark.read.synapsesql("SQLPool01.wwi.TopPurchases")
df2.createTempView("top_purchases_sql")

df2.head(10)

StatementMeta(sparkpool2, 4, 20, Finished, Available)

df2: org.apache.spark.sql.DataFrame = [visitorId: bigint, productId: bigint ... 1 more field]
res8: Array[org.apache.spark.sql.Row] = Array([110458,2384,28], [110459,1470,73], [110460,3181,9], [110460,240,53], [110460,2671,32], [110460,2228,44], [110460,290,95], [110460,1528,2], [110460,578,13], [110460,4472,99])


In [19]:
dfTopPurchasesFromSql = sqlContext.table("top_purchases_sql")

display(dfTopPurchasesFromSql.limit(10))

StatementMeta(sparkpool2, 4, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, 36c2db65-98ad-4b7e-b4cc-07922c7d2fb9)

In [20]:
inner_join = dfsales.join(dfTopPurchasesFromSql,
    (dfsales.CustomerId == dfTopPurchasesFromSql.visitorId) & (dfsales.ProductId == dfTopPurchasesFromSql.productId))

inner_join_agg = (inner_join.select("CustomerId","TotalAmount","Quantity","itemsPurchasedLast12Months","top_purchases_sql.productId")
    .groupBy(["CustomerId","top_purchases_sql.productId"])
    .agg(
        sum("TotalAmount").alias("TotalAmountDecember"),
        sum("Quantity").alias("TotalQuantityDecember"),
        sum("itemsPurchasedLast12Months").alias("TotalItemsPurchasedLast12Months"))
    .orderBy("CustomerId") )

display(inner_join_agg.limit(100))

StatementMeta(sparkpool2, 4, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, ce371ee4-a825-4c9b-899e-1a0b1638f7a5)